In [1]:
import numpy as np
import pandas as pd
from bokeh.layouts import row, widgetbox, column
from bokeh.models import CustomJS, Slider, Select
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import push_notebook, output_notebook, curdoc
from bokeh.client import push_session
output_notebook()

Loading BokehJS ...

In [2]:
df_p = pd.read_csv('../../csvs/initial_orbital_elements.csv', index_col=0)
del df_p['instability_time']
del df_p['Rel_Eerr']
#del df['Rel_Eerr_short']
del df_p["runstring"]
#Nrows = df.shape[0]
#print Nrows, len(np.where(df['Stable']==0)[0])
print df_p.shape
df_p.head()
val_a = df_p.iloc[:200,:] #[:50, :]

(25000, 29)


In [3]:
print val_a.head()

   Stable   RHill12   RHill23     beta12     beta23            m1   a1  \
0       0  0.026033  0.023222   8.508586  15.826967  3.414016e-05  1.0   
1       0  0.022704  0.031284   8.397002  15.367618  2.529838e-07  1.0   
2       0  0.036304  0.032149   3.461288  25.229467  7.380789e-05  1.0   
3       1  0.011704  0.014963  18.513681  19.145889  5.174804e-07  1.0   
4       0  0.006199  0.008634   5.494196   2.928938  5.106746e-07  1.0   

         P1        e1   pomega1    ...       Omega2        f2            m3  \
0  0.999983  0.005031  3.171041    ...     1.573978 -0.567081  1.826900e-06   
1  1.000000  0.010350 -3.564030    ...    -1.749616  1.437355  1.955797e-05   
2  0.999963  0.051912  0.995149    ...    -0.032550 -0.317958  1.477931e-07   
3  1.000000  0.000152 -3.328433    ...     0.946351 -2.294238  1.287851e-06   
4  1.000000  0.001457  1.738448    ...    -0.455684 -1.074927  1.542321e-06   

         a3        P3        e3   pomega3      inc3    Omega3        f3  
0  1.5

In [4]:
# Here is a dict of some keys that I want to be able to pick from for plotting
labels = list(df_p.columns.values)
axis_map = {key:key for key in labels}

#add color and legend
colormap = {1: 'green', 0: 'blue'}
label = {1:"Stable", 0:"Unstable"}
colors = [colormap[x] for x in val_a['Stable']]
labels = [label[x] for x in val_a["Stable"]]

#axis_map.pop("Stable", None)
print axis_map

{'f1': 'f1', 'f2': 'f2', 'f3': 'f3', 'Stable': 'Stable', 'beta23': 'beta23', 'e1': 'e1', 'RHill12': 'RHill12', 'm1': 'm1', 'Omega3': 'Omega3', 'm3': 'm3', 'm2': 'm2', 'e2': 'e2', 'P2': 'P2', 'P3': 'P3', 'P1': 'P1', 'pomega1': 'pomega1', 'pomega2': 'pomega2', 'pomega3': 'pomega3', 'a1': 'a1', 'a3': 'a3', 'a2': 'a2', 'RHill23': 'RHill23', 'Omega1': 'Omega1', 'e3': 'e3', 'Omega2': 'Omega2', 'inc1': 'inc1', 'inc2': 'inc2', 'inc3': 'inc3', 'beta12': 'beta12'}


In [28]:
#This is to update during the callback
#original example in working minimal example
code = ''' //need to resend a lot of the same info to determine the min max values
           //though Id imagine there should be a better way for this. 
           var value1 = val1.value;
           var value2 = val2.value;
           var original_data = original_source.data
           x = original_data[value1];
           y = original_data[value2];
             
           //start and end values with sliders
           var start1 = Math.min.apply(Math, original_data[value1])
           var end1 = Math.max.apply(Math, original_data[value1])
           
           var start2 = Math.min.apply(Math, original_data[value2])
           var end2 = Math.max.apply(Math, original_data[value2])
          
 
           //filter_x = x.filter(inBetween2X);
           //filter_y = y.filter(inBetween2Y);

           var val1 = slide.value
           var val2 = slide2.value
           var val3 = slide3.value
           var val4 = slide4.value
           
           
           function inBetween2X(x) {
               //console.log("F1")
               return (val1 < x & x < va3)

           }
           
           function inBetween2Y(x) {
               //console.log("F2")
               return (val2 < x & x < val4)
       
           }
           
           console.log(val1, val3)
           filter_x = x.filter(inBetween2X)
           console.log("end")
           //console.log(filter_x.length)

           //filter_y = y.filter(inBetween2Y)
           
           '''

code2 = ''' var data = source.data;
            
           //axis values with select widgets
           var value1 = val1.value;
           var value2 = val2.value;
                     
           var original_data = original_source.data
           // get data corresponding to selection
           
           x = original_data[value1];
           y = original_data[value2];
           //data['x'] = x;
           //data['y'] = y;
           
           var start1 = Math.min.apply(Math, original_data[value1])
           var end1 = Math.max.apply(Math, original_data[value1])
           
           var start2 = Math.min.apply(Math, original_data[value2])
           var end2 = Math.max.apply(Math, original_data[value2])
           

           
           //change slider values
           slide.set("start", start1)
           slide.set("end", end1)
           slide.set("step", (end1 -start1)/20.)
           //slide.set("value", start1)
           slide.set("title", val1)
           //slide.set("value", slide.get('start'))
                      
           slide2.set("start", start2)
           slide2.set("end", end2)
           slide2.set("step", (end2 -start2)/20.)
           slide2.set("title", val2)
           
           slide3.set("start", start1)
           slide3.set("end", end1)
           slide3.set("step", (end1 -start1)/20.)
           slide3.set("title", val1+"end")
           
           slide4.set("start", start2)
           slide4.set("end", end2)
           slide4.set("step", (end2 -start2)/20.)
           slide4.set("title", val2+"end")          
          
           source.trigger('change');
           // set axis labels
           x_axis.axis_label = value1;
           y_axis.axis_label = value2;
           //slide3.set("value", end1);
           
           // #need to create filter based on min and max vals
           //x = original_data[value1];

           
           var val1 = slide.value
           var val2 = slide2.value
           var val3 = slide3.value
           var val4 = slide4.value
           
           function inBetween2X(x) {
               //console.log("F1")
               return (val1 < x & x < val3)

           }
           
           function inBetween2Y(x) {
               //console.log("F2")
               return (val2 < x & x<val4)
       
           }
           
           
           filter_x = x.filter(inBetween2X);
           filter_y = y.filter(inBetween2Y);
                  
           data['x'] = filter_x;
           data['y'] = filter_y;
                  
           console.log(filter_y.length)
           console.log(filter_x.length)
            
           console.log("Code Syntax is atleast correct")
           //console.log(len(filter_x))
           
           '''


source = ColumnDataSource(data=dict(x=val_a['RHill12'], y=val_a['RHill23'], label = labels, ))
original_source = ColumnDataSource(data=val_a.to_dict(orient='list'))

#plot the figures
plot = figure(plot_width=400, plot_height=400)
plot.circle(x= "x",y="y", source=source, line_width=3, line_alpha=0.6, color = colors, legend = 'label')

slider = Slider(title ="Slider X Start", start=df_p['RHill12'].min(), end=df_p['RHill12'].max(),
                step=(df_p['RHill12'].max() - df_p['RHill12'].min()) /10., 
               value = df_p['RHill12'].min())
slider2 = Slider(title ="Slider Y Start", start=df_p['RHill23'].min(), end=df_p['RHill23'].max(),
                 step=(df_p['RHill23'].max() - df_p['RHill23'].min())/10., 
                 value = df_p['RHill23'].min())


slider3 = Slider(title ="Slider X End", start=df_p['RHill12'].min(), end=df_p['RHill12'].max(),
                step=(df_p['RHill12'].max() - df_p['RHill12'].min()) /10., 
               value = df_p['RHill12'].max())
slider4 = Slider(title ="Slider Y End", start=df_p['RHill23'].min(), end=df_p['RHill23'].max(),
                 step=(df_p['RHill23'].max() - df_p['RHill23'].min())/10., 
                 value = df_p['RHill23'].max())


callback = CustomJS(args=dict(source=source, original_source = original_source,
                              x_axis=plot.xaxis[0],y_axis=plot.yaxis[0], slide = slider ,
                              slide2 = slider2, slider3 =slider3,
                              slider4 = slider4), code=code2)



#create a separate callback to manage chaning data
callbackDRange = CustomJS(args = dict(source = source,original_source = original_source,
                              x_axis=plot.xaxis[0],y_axis=plot.yaxis[0], slide = slider ,
                              slide2 = slider2, slider3 =slider3,
                              slider4 = slider4), code=code2 )

slider.callback = callback
callback.args["slide"] = slider
callbackDRange.args["slide"] = slider


slider2.callback = callback
callback.args["slide2"] = slider2
callbackDRange.args["slide2"] = slider2

slider3.callback = callback
callback.args["slide3"] = slider3
callbackDRange.args["slide3"] = slider3

slider4.callback = callback
callback.args["slide4"] = slider4
callbackDRange.args["slide4"] = slider4

#call this when the value on the slider changes
#slider.js_on_change('value', callbackDRange)
#slider2.js_on_change('value', callbackDRange)
#slider3.js_on_change('value', callbackDRange)
#lider4.js_on_change('value', callbackDRange)



#Create two select widgets to pick the features of interest 
x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="RHill12", callback = callback)
callback.args["val1"] = x_axis
callbackDRange.args["val1"]= x_axis

y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="RHill23", callback = callback)
callback.args["val2"] = y_axis
callbackDRange.args["val2"]= y_axis

plot.xaxis[0].axis_label = 'RHill12'
plot.yaxis[0].axis_label = 'RHill23'
#Error message due to legend, need to find a better way to do so

/mnt/raid-cita/nhussain/venv-2.7.13/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/mnt/raid-cita/nhussain/venv-2.7.13/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [29]:
#Display the graph in a jupyter notebook
layout = column(plot, x_axis, y_axis, row(slider, slider2),row(slider3, slider4) )
show(layout, notebook_handle=True)